In [30]:
PREPROCESSED_SEQUENCE_PATH = '/home/bandeja/datasets/samsung-spatial/processed/2022-04-16-12-17-44_round1/'

In [31]:
import cv2
import imageio
import numpy as np
import os
import scipy.ndimage
import shutil
from tqdm import tqdm

In [32]:
camera_folders = [name for name in os.listdir(PREPROCESSED_SEQUENCE_PATH) if os.path.isdir(os.path.join(PREPROCESSED_SEQUENCE_PATH, name))]
camera_folders

['5s', '3m']

In [33]:
for camera_folder in tqdm(camera_folders):
    one_camera_data_path = os.path.join(PREPROCESSED_SEQUENCE_PATH, camera_folder)

    bf_postprocessed_path = os.path.join(one_camera_data_path, 'bf')

    if os.path.exists(bf_postprocessed_path):
        shutil.rmtree(bf_postprocessed_path)

    os.mkdir(bf_postprocessed_path)
    
    with open(os.path.join(one_camera_data_path, 'associations_dense.txt'), 'r') as association_file:
        lines = association_file.readlines()
        for i, line in enumerate(lines):
            _, color_image_rel_path, _, depth_image_rel_path = line.strip().split(' ')

            color_image = cv2.imread(os.path.join(one_camera_data_path, color_image_rel_path))
            cv2.imwrite(os.path.join(bf_postprocessed_path, 'frame-%06d.color.jpg' % i), color_image)

            depth_image = imageio.imread(os.path.join(one_camera_data_path, depth_image_rel_path))
            imageio.imwrite(os.path.join(bf_postprocessed_path, 'frame-%06d.depth.png' % i), depth_image // 5)

100%|██████████| 2/2 [16:09<00:00, 485.00s/it]


In [34]:
for camera_folder in tqdm(camera_folders):
    one_camera_data_path = os.path.join(PREPROCESSED_SEQUENCE_PATH, camera_folder)
    K_undist = np.load(os.path.join(one_camera_data_path, 'camera_params.npy'))

    new_bundling_config_path = os.path.join(one_camera_data_path, 'bfBundlingConfig.txt')
    shutil.copyfile('configs/bfBundlingConfig.txt', new_bundling_config_path)

    new_params_config_path = os.path.join(one_camera_data_path, 'bfParametersConfig.txt')
    shutil.copyfile('configs/bfParametersConfig.txt', new_params_config_path)

    with open(new_params_config_path, 'r') as f:
        lines = f.readlines()
        lines[117] = 's_cameraIntrinsicFx = {};\n'.format(K_undist[0, 0])
        lines[118] = 's_cameraIntrinsicFy = {};\n'.format(K_undist[1, 1])
        lines[119] = 's_cameraIntrinsicCx = {};\n'.format(K_undist[0, 2])
        lines[120] = 's_cameraIntrinsicCy = {};\n'.format(K_undist[1, 2])
        
    with open(new_params_config_path, 'w') as f:
        f.writelines(lines)

100%|██████████| 2/2 [00:00<00:00, 712.47it/s]
